https://coin-or.github.io/pulp/CaseStudies/a_set_partitioning_problem.html

In [1]:
%load_ext nb_black

import pandas as pd

from pulp import LpProblem, LpStatus, LpVariable, lpSum, value, allcombinations
from pulp import LpBinary, LpMinimize, LpMaximize

<IPython.core.display.Javascript object>

In [2]:
MAX_TABLES = 5
MAX_TABLE_SIZE = 4
GUESTS = "A B C D E F G I J K L M N O P Q R".split()

<IPython.core.display.Javascript object>

In [3]:
def happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(ord(table[0]) - ord(table[-1]))

<IPython.core.display.Javascript object>

In [4]:
# create list of all possible tables
GUEST_COMBINATIONS = [tuple(c) for c in allcombinations(GUESTS, MAX_TABLE_SIZE)]

<IPython.core.display.Javascript object>

In [5]:
# create a binary variable to state that a table setting is used
tables = LpVariable.dicts(
    "table", GUEST_COMBINATIONS, lowBound=0, upBound=1, cat="Integer"
)

<IPython.core.display.Javascript object>

In [6]:
model = LpProblem("Wedding Seating Model", LpMaximize)

model += lpSum([happiness(combo) * tables[combo] for combo in GUEST_COMBINATIONS])

# specify the maximum number of tables
model += (
    lpSum([tables[combo] for combo in GUEST_COMBINATIONS]) <= MAX_TABLES,
    "Maximum_number_of_tables",
)

# A guest must seated at one and only one table
for guest in GUESTS:
    model += (
        lpSum([tables[combo] for combo in GUEST_COMBINATIONS if guest in combo]) == 1,
        "Must_seat_%s" % guest,
    )

/usr/local/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


<IPython.core.display.Javascript object>

In [7]:
model.solve()

LpStatus[model.status]

'Optimal'

<IPython.core.display.Javascript object>

In [8]:
print("The choosen tables are out of a total of %s:" % len(GUEST_COMBINATIONS))
for guests in GUEST_COMBINATIONS:
    if tables[guests].varValue == 1.0:
        print(guests)

The choosen tables are out of a total of 3213:
('A', 'M', 'Q')
('B', 'F', 'P')
('C', 'L', 'R')
('D', 'G', 'J', 'N')
('E', 'I', 'K', 'O')


<IPython.core.display.Javascript object>